In [1]:
from pyspark import SparkConf, SparkContext,SQLContext  
from pyspark.sql import SparkSession   
from pyspark.ml.feature import Word2Vec,CountVectorizer  
from pyspark.ml.clustering import LDA, LDAModel  
from pyspark.sql.functions import col, udf  
from pyspark.sql.types import IntegerType,ArrayType,StringType  
import pylab as pl 

In [2]:
def to_word(termIndices):
  words = []  
  for termID in termIndices:
    words.append(vocab_broadcast.value[termID])      
  return words

In [3]:
#Load your document dataframe here
#================your code here==================
spark = SparkSession.builder.appName("LDA").getOrCreate()
data = sc.textFile('gs://yuqintwitter/tweets.txt')
words = data.map(lambda line: line.split(" "))
words = words.map(lambda line: [word for word in line if (word.isalpha() and word != '' and word !='Oct')])
filt = words.filter(lambda line: len(line) > 5).zipWithIndex()
df = spark.createDataFrame(filt, ["words", "NO."])
df.show()

21/10/24 23:31:05 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
21/10/24 23:31:05 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #0,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at c

+--------------------+---+
|               words|NO.|
+--------------------+---+
|[I, can, interpre...|  0|
|[Dans, ces, vraim...|  1|
|[dire, que, pas, ...|  2|
|[illustration, to...|  3|
|[feeling, like, b...|  4|
|[eu, une, soudain...|  5|
|[Do, you, want, t...|  6|
|[for, rt, Dec, fo...|  7|
|[i, was, in, a, h...|  8|
|[en, los, que, me...|  9|
|[Il, fotografo, t...| 10|
|[interest, check,...| 11|
|[atingi, minha, e...| 12|
|[it, stands, so, ...| 13|
|[Exacto, a, los, ...| 14|
|[wsh, mes, si, vo...| 15|
|[Je, rien, faire,...| 16|
|[movie, marathon,...| 17|
|[Sheds, Tears, In...| 18|
|[vcs, sabem, pq, ...| 19|
+--------------------+---+
only showing top 20 rows



In [4]:
#CountVectorizer
#================your code here==================
Vec = CountVectorizer(inputCol='words', outputCol="features")
Model=Vec.fit(df)
cvResult = Model.transform(df)
cvResult.show()
#==================================================

+--------------------+---+--------------------+
|               words|NO.|            features|
+--------------------+---+--------------------+
|[I, can, interpre...|  0|(8319,[0,2,3,6,7,...|
|[Dans, ces, vraim...|  1|(8319,[0,11,15,16...|
|[dire, que, pas, ...|  2|(8319,[1,2,3,8,13...|
|[illustration, to...|  3|(8319,[3,4,14,16,...|
|[feeling, like, b...|  4|(8319,[0,4,30,36,...|
|[eu, une, soudain...|  5|(8319,[0,1,9,11,1...|
|[Do, you, want, t...|  6|(8319,[0,2,4,10,1...|
|[for, rt, Dec, fo...|  7|(8319,[0,34,36,50...|
|[i, was, in, a, h...|  8|(8319,[2,3,4,5,10...|
|[en, los, que, me...|  9|(8319,[0,1,5,8,9,...|
|[Il, fotografo, t...| 10|(8319,[0,2,9,19,2...|
|[interest, check,...| 11|(8319,[0,44,68,64...|
|[atingi, minha, e...| 12|(8319,[0,1,8,12,3...|
|[it, stands, so, ...| 13|(8319,[0,3,4,5,6,...|
|[Exacto, a, los, ...| 14|(8319,[0,1,2,8,9,...|
|[wsh, mes, si, vo...| 15|(8319,[0,1,2,13,1...|
|[Je, rien, faire,...| 16|(8319,[0,1,9,11,1...|
|[movie, marathon,...| 17|(8319,[0,3,10,

In [5]:
#train LDA model, cluster the documents into 10 topics 
#================your code here==================
lda=LDA(k=10,maxIter=10)
ldaModel=lda.fit(cvResult)
#==================================================

21/10/24 23:31:32 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1430.6 KiB
21/10/24 23:31:33 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1430.6 KiB
21/10/24 23:31:34 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1430.6 KiB
21/10/24 23:31:34 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1430.6 KiB
21/10/24 23:31:34 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1430.6 KiB
21/10/24 23:31:35 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1430.6 KiB
21/10/24 23:31:35 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1430.6 KiB
21/10/24 23:31:35 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1430.6 KiB
21/10/24 23:31:35 WARN org.apache.spark.scheduler.DAGScheduler: Broadcas

In [8]:
transformed = ldaModel.transform(cvResult).select("topicDistribution")  
#show the weight of every topic Distribution 
transformed.show(truncate=False)  

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topicDistribution                                                                                                                                                                                                        |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.0016313719331940973,0.9865506283684491,0.0014559686467718602,0.0014753535960412704,0.00145574231035269,0.0014615575568711699,0.001448455373462369,0.0014549411600922532,0.001595172402703231,0.0014708086520618523]   |
|[0.9625080503008433,0.02438292027600502,0.0016150110512988412,0.0016365133274243038,0.001614760150372884,0.001621190649

In [9]:
#The higher ll is, the lower lp is, the better model is.
ll = ldaModel.logLikelihood(cvResult)  
lp = ldaModel.logPerplexity(cvResult)
print("ll: ", ll)
print("lp: ", lp)

ll:  -456479.0898274858
lp:  9.192643329792089


In [10]:
# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())+ " words):")
topics = ldaModel.topicsMatrix()
print(topics)

Learned topics (as distributions over vocab of 8319 words):
DenseMatrix([[ 87.55556823, 230.89779267,   5.67454723, ...,   6.21162192,
               84.09128382,  13.21723224],
             [213.17455533,   2.59655918,   0.67055143, ...,   1.71140612,
               48.29509383,  12.47531816],
             [ 35.36272476, 132.16231343,   6.19347975, ...,   1.14798563,
               32.90836051,   0.77491769],
             ...,
             [  0.7206478 ,   1.23221094,   0.82918947, ...,   0.69832621,
                0.77330948,   0.68028402],
             [  0.75865071,   0.74687932,   0.7545155 , ...,   0.69650978,
                0.77690882,   0.67792634],
             [  0.74532152,   0.76233651,   0.71099719, ...,   0.88514581,
                0.86905788,   0.67784328]])
